In [ ]:
import cv2
import numpy as np
import os

# --- Configuration ---
main_image_path = 'barbara.jpg'
# !!! CHANGE: Update the patch image path !!!
patch_image_path = 'patch_1.png'
# !!! CHANGE: Update the output image path !!!
output_image_path = 'barbara_with_patch_1_box.jpg' # Changed output name

# --- Function to calculate Euclidean distance ---
def calculate_euclidean_distance(patch1, patch2):
    """Calculates the Euclidean distance between two image patches."""
    # Ensure patches have the same shape
    if patch1.shape != patch2.shape:
        raise ValueError("Patches must have the same dimensions and channels.")

    # Using squared Euclidean distance for comparison (avoids costly sqrt)
    # Convert to float to prevent overflow issues with uint8 subtraction
    dist_sq = np.sum((patch1.astype(np.float64) - patch2.astype(np.float64))**2)
    return dist_sq

# --- Main Script ---

# 1. Load Images
print(f"Loading main image: {main_image_path}")
if not os.path.exists(main_image_path):
    print(f"Error: Main image file not found at {main_image_path}")
    exit()
main_img = cv2.imread(main_image_path)
if main_img is None:
    print(f"Error: Could not load main image {main_image_path}")
    exit()

print(f"Loading patch image: {patch_image_path}") # Will now print patch_1.png
if not os.path.exists(patch_image_path):
    print(f"Error: Patch image file not found at {patch_image_path}")
    # Provide a helpful hint if the specific file is missing
    print(f"Hint: Make sure '{patch_image_path}' is in the same folder as the script.")
    exit()
patch_img = cv2.imread(patch_image_path)
if patch_img is None:
    print(f"Error: Could not load patch image {patch_image_path}")
    exit()

# --- Add check for grayscale ---
# If barbara is grayscale but patch isn't, or vice-versa, processing might fail
# Let's ensure consistency. Common case: barbara is grayscale.
if len(main_img.shape) == 2 and len(patch_img.shape) == 3:
     print("Warning: Main image is grayscale, patch is color. Converting patch to grayscale.")
     patch_img = cv2.cvtColor(patch_img, cv2.COLOR_BGR2GRAY)
elif len(main_img.shape) == 3 and len(patch_img.shape) == 2:
     print("Warning: Main image is color, patch is grayscale. Converting patch to grayscale requires converting main image too.")
     # Decide on a strategy: convert main image OR convert patch to color (less accurate potentially)
     # Option 1: Convert main to gray (consistent with likely barbara.jpg source)
     print("Converting main image to grayscale for comparison.")
     main_img = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
     # Option 2: Convert patch to color (if you need color matching)
     # patch_img = cv2.cvtColor(patch_img, cv2.COLOR_GRAY2BGR)

# Re-check dimensions after potential conversion
main_h, main_w = main_img.shape[:2]
patch_h, patch_w = patch_img.shape[:2]

print(f"Main image dimensions (HxW): {main_h} x {main_w}")
print(f"Patch image dimensions (HxW): {patch_h} x {patch_w}")

# Check if patch is larger than the main image
if patch_h > main_h or patch_w > main_w:
    print("Error: Patch image is larger than the main image.")
    exit()

# 3. Search for the Patch (Sliding Window)
min_distance = np.inf
best_match_coords = (0, 0) # (y, x) of top-left corner

print("Searching for the best match using Euclidean distance...")
# Iterate through all possible top-left corners (y, x) for the patch
for y in range(main_h - patch_h + 1):
    for x in range(main_w - patch_w + 1):
        # Extract the current window/candidate patch from the main image
        candidate_patch = main_img[y : y + patch_h, x : x + patch_w]

        # Calculate the distance (using squared Euclidean for efficiency)
        distance = calculate_euclidean_distance(patch_img, candidate_patch)

        # Update if this is a better match
        if distance < min_distance:
            min_distance = distance
            best_match_coords = (y, x) # Store NumPy-style coordinates (row, col)

# 4. Get Coordinates for Drawing
best_y, best_x = best_match_coords
top_left = (best_x, best_y)         # OpenCV uses (x, y) for drawing
bottom_right = (best_x + patch_w, best_y + patch_h)

print(f"Best match found for '{patch_image_path}'!") # Added filename here
print(f" -> Minimum Squared Euclidean Distance: {min_distance:.2f}")
print(f" -> Top-left corner (x, y): {top_left}")
print(f" -> Bottom-right corner (x, y): {bottom_right}")

# 5. Draw Bounding Box
# Create a copy to draw on, preserving the original
# Need to handle drawing on color or grayscale
if len(main_img.shape) == 3:
    output_img = main_img.copy()
    box_color = (0, 0, 255) # BGR format for Red
else: # Grayscale image
    # Convert grayscale back to BGR *for drawing* the colored box
    output_img = cv2.cvtColor(main_img, cv2.COLOR_GRAY2BGR)
    box_color = (0, 0, 255) # BGR format for Red (still works)

box_thickness = 2
cv2.rectangle(output_img, top_left, bottom_right, box_color, box_thickness)

# 6. Save and Display Result
print(f"Saving result with bounding box to: {output_image_path}")
cv2.imwrite(output_image_path, output_img)

print("Displaying result. Press any key to close.")
# Load original main image again for consistent display if it was converted
main_img_display = cv2.imread(main_image_path) 
cv2.imshow('Main Image', main_img_display)
cv2.imshow('Patch (patch_1.png)', patch_img) # Updated window title
cv2.imshow('Found Patch Location', output_img)

# Wait for a key press and then close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Done.")